In [ ]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET  # for parsing XML
import numpy as np  # for using pandas
import pandas as pd  # for using dataframes1
import xmltodict, json
import pprint

In [ ]:
def extract_film_info(url):
    req = (requests.get(url))
    dict_= xmltodict.parse(req.content) 
    comment_list = dict_['items']['item']['link']
    df = pd.DataFrame(comment_list)
    df = df[(df['@type'].str.contains('category')) | (df['@type'].str.contains('mechanic'))]
    df = df.pivot(columns = '@type', values='@value')
    df['boardgame_title']= dict_['items']['item']['name'][0]['@value']
    return df

In [ ]:
def extract_comment_rating(link):
    req = (requests.get(link))
    dict_data= xmltodict.parse(req.content) 
    comment_list = dict_data['items']['item']['comments']['comment']
    df = pd.DataFrame(comment_list)
    return df

In [ ]:
def clean_data(df, url_info):
    df = df.rename({'@rating': 'rating', '@value': 'value'}, axis=1)
    df['value'].replace('', np.nan, inplace=True)
    df.dropna(subset=['value'], inplace=True)
    req = (requests.get(url_info)) 
    dict_data= xmltodict.parse(req.content) 
    df['boardgame_title']= dict_data['items']['item']['name'][0]['@value']
    return df

In [ ]:
def return_dataset(url_info, url_commento):
    film_info = extract_film_info(url_info)
    list_comments = extract_comment_rating(url_commento)
    film_comments = clean_data(list_comments, url_info)
    data = pd.merge(film_comments, film_info,on = 'boardgame_title' ,how = 'outer')
    final_dataset = data[['@username','rating', 'value', 'boardgame_title', 'boardgamecategory', 'boardgamemechanic']]
    return final_dataset